# Stock Trading System

## Setting up the environment

### Get the requirements

In [1]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=2185e2c1ccc71480c2cf47cea82344e8a1330b65c06641ac76d540bd5120a176
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


### Config

In [1]:
# List of 50 popular stock symbols
POPULAR_STOCKS = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "JPM", "JNJ", "V",
    "PG", "DIS", "MA", "PYPL", "NFLX", "ADBE", "INTC", "CMCSA", "PFE", "KO",
    "PEP", "CSCO", "XOM", "ABT", "CRM", "NKE", "MRK", "WMT", "T", "BAC",
    "MCD", "COST", "CVX", "MDT", "NEE", "LLY", "HON", "ORCL", "AVGO", "TXN",
    "UNH", "QCOM", "BMY", "IBM", "AMD", "AMAT", "GE", "CAT", "MMM", "GS"
]

# Use last 180 days of stock data to predict the next day
SEQ_LENGTH = 180

### Imports

In [2]:
import yfinance as yf
import pandas as pd
import os
import ta
from datetime import datetime
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import save_model
import joblib


In [5]:
os.getcwd()

'/content'

## Data Pipeline

### Fetch Stock Data

In [6]:
DATA_PATH = "data_pipeline/raw_data"

def fetch_and_store_stock_data(stock_symbol):
  """
  Fetches historical stock data from Yahoo Finance, applies technical indicators,
  and saves the processed data as a CSV file.
  """
  print(f"Fetching data for {stock_symbol}...")

  stock = yf.Ticker(stock_symbol)
  df = stock.history(period="5y")

  if df.empty:
    print(f"No data found for {stock_symbol}. Skipping.")
    return

  # Keep only useful columns
  df = df[['Close', 'Open', 'High', 'Low', 'Volume']]

  # Compute Technical Indicators using `ta` library
  df["SMA_20"] = ta.trend.sma_indicator(df["Close"], window=20)
  df["SMA_50"] = ta.trend.sma_indicator(df["Close"], window=50)
  df["EMA_20"] = ta.trend.ema_indicator(df["Close"], window=20)
  df["RSI"] = ta.momentum.rsi(df["Close"], window=14)
  df["MACD"] = ta.trend.macd(df["Close"])
  df["Bollinger_Upper"] = ta.volatility.bollinger_hband(df["Close"], window=20)
  df["Bollinger_Lower"] = ta.volatility.bollinger_lband(df["Close"], window=20)

  df.dropna(inplace=True)  # Remove rows with NaN values

  # Add timestamp for logging
  df["Fetched_Date"] = datetime.today().strftime('%Y-%m-%d')

  # Save to CSV
  os.makedirs(DATA_PATH, exist_ok=True)
  file_path = os.path.join(DATA_PATH, f"{stock_symbol}.csv")
  df.to_csv(file_path)
  print(f"Data saved: {file_path}")

# Run for all stocks in `config.py`
for stock in POPULAR_STOCKS:
  fetch_and_store_stock_data(stock)

print("All stock data fetched and stored successfully!")


Fetching data for AAPL...
Data saved: data_pipeline/raw_data/AAPL.csv
Fetching data for MSFT...
Data saved: data_pipeline/raw_data/MSFT.csv
Fetching data for GOOGL...
Data saved: data_pipeline/raw_data/GOOGL.csv
Fetching data for AMZN...
Data saved: data_pipeline/raw_data/AMZN.csv
Fetching data for TSLA...
Data saved: data_pipeline/raw_data/TSLA.csv
Fetching data for META...
Data saved: data_pipeline/raw_data/META.csv
Fetching data for NVDA...
Data saved: data_pipeline/raw_data/NVDA.csv
Fetching data for JPM...
Data saved: data_pipeline/raw_data/JPM.csv
Fetching data for JNJ...
Data saved: data_pipeline/raw_data/JNJ.csv
Fetching data for V...
Data saved: data_pipeline/raw_data/V.csv
Fetching data for PG...
Data saved: data_pipeline/raw_data/PG.csv
Fetching data for DIS...
Data saved: data_pipeline/raw_data/DIS.csv
Fetching data for MA...
Data saved: data_pipeline/raw_data/MA.csv
Fetching data for PYPL...
Data saved: data_pipeline/raw_data/PYPL.csv
Fetching data for NFLX...
Data saved: 

### Preprocess Data

In [7]:
RAW_DATA_PATH = "data_pipeline/raw_data"
DATA_PATH = "data_pipeline/processed_data"

def preprocess_stock_data():
  """
  Loads, cleans, and processes all stock data CSVs in `data_pipeline/data/`
  - Handles missing values (technical indicators can sometimes have NaNs)
  - Ensures all stocks have the same historical length
  - Saves the processed data back to `data_pipeline/data/`
  """

  print("Preprocessing stock data...")

  processed_data = {}

  # Load each stock's CSV file
  for stock in POPULAR_STOCKS:
    file_path = os.path.join(RAW_DATA_PATH, f"{stock}.csv")

    if not os.path.exists(file_path):
      print(f"⚠ {stock}: No data file found. Skipping...")
      continue

    df = pd.read_csv(file_path, index_col="Date", parse_dates=True)

    # Step 1: Handle Missing Values
    df.fillna(method="ffill", inplace=True)  # Forward fill missing values
    df.fillna(method="bfill", inplace=True)  # Backward fill if needed
    df.dropna(inplace=True)  # Drop any remaining NaNs

    # Step 2: Trim all stocks to the same historical length
    processed_data[stock] = df

  if not processed_data:
    raise ValueError("No stock data found. Ensure `fetch_stock_data.py` ran successfully.")

  # Step 3: Ensure all stocks have the same length (trim to the shortest)
  min_length = min(len(df) for df in processed_data.values())

  for stock in processed_data:
    processed_data[stock] = processed_data[stock].tail(min_length)  # Trim each stock's data

  # Step 4: Save Cleaned Data
  os.makedirs(DATA_PATH, exist_ok=True)
  for stock, df in processed_data.items():
    file_path = os.path.join(DATA_PATH, f"{stock}.csv")
    df.to_csv(file_path)
    print(f"{stock}: Data preprocessed and saved ({len(df)} rows).")

  print("All stock data successfully preprocessed!")

# Run preprocessing
if __name__ == "__main__":
  preprocess_stock_data()


Preprocessing stock data...


<ipython-input-7-afd42ddb2a69>:27: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)  # Forward fill missing values
<ipython-input-7-afd42ddb2a69>:28: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="bfill", inplace=True)  # Backward fill if needed
<ipython-input-7-afd42ddb2a69>:27: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)  # Forward fill missing values
<ipython-input-7-afd42ddb2a69>:28: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="bfill", inplace=True)  # Backward fill if needed
<ipython-input-7-afd42ddb2a69>:2

AAPL: Data preprocessed and saved (1207 rows).
MSFT: Data preprocessed and saved (1207 rows).
GOOGL: Data preprocessed and saved (1207 rows).
AMZN: Data preprocessed and saved (1207 rows).
TSLA: Data preprocessed and saved (1207 rows).
META: Data preprocessed and saved (1207 rows).
NVDA: Data preprocessed and saved (1207 rows).
JPM: Data preprocessed and saved (1207 rows).
JNJ: Data preprocessed and saved (1207 rows).
V: Data preprocessed and saved (1207 rows).
PG: Data preprocessed and saved (1207 rows).
DIS: Data preprocessed and saved (1207 rows).
MA: Data preprocessed and saved (1207 rows).
PYPL: Data preprocessed and saved (1207 rows).
NFLX: Data preprocessed and saved (1207 rows).
ADBE: Data preprocessed and saved (1207 rows).
INTC: Data preprocessed and saved (1207 rows).
CMCSA: Data preprocessed and saved (1207 rows).
PFE: Data preprocessed and saved (1207 rows).
KO: Data preprocessed and saved (1207 rows).
PEP: Data preprocessed and saved (1207 rows).
CSCO: Data preprocessed a

### Data loader

In [8]:
DATA_PATH = "data_pipeline/processed_data"
SCALER_PATH = "data_pipeline/scalers"

def load_stock_data(stock_symbol=None):
  """
  Load and preprocess stock data.
  - If `stock_symbol` is provided, loads and scales data for a single stock.
  - If `stock_symbol` is None, loads and scales data for all 50 stocks.
  """

  os.makedirs(SCALER_PATH, exist_ok=True)  # Ensure scaler directory exists

  if stock_symbol:
    file_path = os.path.join(DATA_PATH, f"{stock_symbol}.csv")
    if not os.path.exists(file_path):
      raise ValueError(f"No stock data found for {stock_symbol}. Run `fetch_stock_data.py` first.")

    df = pd.read_csv(file_path, index_col="Date", parse_dates=True)

    # Drop non-numeric columns
    if "Fetched_Date" in df.columns:
      df.drop(columns=["Fetched_Date"], inplace=True)

    # Ensure all values are numeric
    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(inplace=True)

    # Select all 12 features for training
    selected_columns = ['Close', 'Open', 'High', 'Low', 'Volume', 'SMA_20', 'SMA_50', 'EMA_20', 'RSI', 'MACD', 'Bollinger_Upper', 'Bollinger_Lower']
    df = df[selected_columns].copy()

    # Convert all values to float32
    df = df.astype(np.float32)

    # Normalize all features
    scaler = MinMaxScaler()
    df.loc[:, selected_columns] = scaler.fit_transform(df[selected_columns])

    # Save the scaler
    joblib.dump(scaler, os.path.join(SCALER_PATH, f"{stock_symbol}_scaler.pkl"))

    stock_data = df.values

    # Ensure enough data is available
    if len(stock_data) < SEQ_LENGTH + 20:
      raise ValueError(f"Not enough data for {stock_symbol}. Need at least {SEQ_LENGTH + 20} days.")

    # Create sequences
    X, y = [], []
    for i in range(len(stock_data) - SEQ_LENGTH - 20 + 1):
      x_seq = stock_data[i:i + SEQ_LENGTH, :]
      y_seq = stock_data[i + SEQ_LENGTH:i + SEQ_LENGTH + 20, :5]  # Include 5 features in y

      if x_seq.shape == (SEQ_LENGTH, stock_data.shape[1]) and y_seq.shape == (20, 5):
        X.append(x_seq)
        y.append(y_seq)

    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)  # Ensure y shape is (samples, 20, 5)

    print(f"Loaded {len(X)} samples for {stock_symbol}, Shape of X: {X.shape}, Shape of y: {y.shape}")
    return X, y, scaler

  else:
    # Load all stocks
    data_dict = {}
    scalers = {}

    for stock in POPULAR_STOCKS:
      file_path = os.path.join(DATA_PATH, f"{stock}.csv")
      if os.path.exists(file_path):
        df = pd.read_csv(file_path, index_col="Date", parse_dates=True)

        # Drop non-numeric columns
        if "Fetched_Date" in df.columns:
          df.drop(columns=["Fetched_Date"], inplace=True)

        # Ensure all values are numeric
        df = df.apply(pd.to_numeric, errors='coerce')
        df.dropna(inplace=True)

        # Select all 12 features for training
        selected_columns = ['Close', 'Open', 'High', 'Low', 'Volume', 'SMA_20', 'SMA_50', 'EMA_20', 'RSI', 'MACD', 'Bollinger_Upper', 'Bollinger_Lower']
        df = df[selected_columns].copy()

        # Convert all values to float32
        df = df.astype(np.float32)

        # Normalize all features
        scaler = MinMaxScaler()
        df.loc[:, selected_columns] = scaler.fit_transform(df[selected_columns])

        # Save scaler for this stock
        joblib.dump(scaler, os.path.join(SCALER_PATH, f"{stock}_scaler.pkl"))

        data_dict[stock] = df
        scalers[stock] = scaler

    if not data_dict:
      raise ValueError("No stock data found. Run `fetch_stock_data.py` first.")

    # Convert all stock data to a single NumPy array
    stock_data = np.array([data_dict[stock].values for stock in POPULAR_STOCKS])

    # Ensure sequences are correctly formatted
    X, y = [], []
    for i in range(len(stock_data[0]) - SEQ_LENGTH - 20 + 1):
      X.append(stock_data[:, i:i + SEQ_LENGTH, :])
      y.append(stock_data[:, i + SEQ_LENGTH:i + SEQ_LENGTH + 20, :5])  # Include 5 features in y

    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)

    print(f"Loaded {X.shape[0]} sequences for all stocks, Shape of X: {X.shape}, Shape of y: {y.shape}")
    return X, y, scalers

## Model Training Pipeline

### Train Hybrid Model

In [5]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detect TPU
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.TPUStrategy(tpu)  # TPU Strategy
  print("TPU Detected and Initialized")
except:
  strategy = tf.distribute.get_strategy()
  print("TPU Not Detected, Using CPU/GPU Instead")

TPU Not Detected, Using CPU/GPU Instead


In [10]:
MODEL_PATH = "trained_models/"

def build_hybrid_model(input_shape):
  """
  Builds a hybrid model combining CNN, LSTM, and Transformer layers.
  """
  with strategy.scope():
    inputs = tf.keras.Input(shape=input_shape)

    # CNN Feature Extraction
    x = tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)

    # LSTM for Sequential Learning
    x = tf.keras.layers.LSTM(128, return_sequences=True)(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.LSTM(64, return_sequences=True)(x)

    # Transformer Attention
    x = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)

    # Flatten Output Before Dense Layers
    x = tf.keras.layers.GlobalAveragePooling1D()(x)

    # Dense Layers
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(20 * 5)(x)  # Predicts next 20 days

    model = tf.keras.Model(inputs, outputs)
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, epsilon=1e-7, clipnorm=1.0)
    model.compile(optimizer=optimizer, loss="mean_squared_error")

  return model

def train_hybrid_model():
  """
  Trains a single hybrid model on TPU for all 50 stocks.
  """

  # Load data for all stocks
  X, y, scalers = load_stock_data()

  # Reshape input to merge stocks into the batch dimension
  num_samples, num_stocks, seq_len, num_features = X.shape
  X = X.reshape(num_samples * num_stocks, seq_len, num_features)
  y = y.reshape(num_samples * num_stocks, 20, 5)
  y = y.reshape(num_samples * num_stocks, 20 * 5)

  print(f"Final Shape Before Training - X: {X.shape}, y: {y.shape}")

  # Build model inside TPU/GPU scope
  model = build_hybrid_model(input_shape=(SEQ_LENGTH, X.shape[-1]))

  # Train the model on TPU
  model.fit(X, y, epochs=30, batch_size=64)

  # Save the model using best practices
  os.makedirs(MODEL_PATH, exist_ok=True)
  save_model(model, f"{MODEL_PATH}/hybrid_model.keras")
  print(f"Hybrid Model saved in `{MODEL_PATH}/hybrid_model.keras`.")

if __name__ == "__main__":
  train_hybrid_model()


Loaded 1008 sequences for all stocks, Shape of X: (1008, 50, 180, 12), Shape of y: (1008, 50, 20, 5)
Final Shape Before Training - X: (50400, 180, 12), y: (50400, 100)
Epoch 1/30
788/788 ━━━━━━━━━━━━━━━━━━━━ 23s 19ms/step - loss: 0.1446
Epoch 2/30
788/788 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - loss: 0.0261
Epoch 3/30
788/788 ━━━━━━━━━━━━━━━━━━━━ 17s 22ms/step - loss: 0.0182
Epoch 4/30
788/788 ━━━━━━━━━━━━━━━━━━━━ 17s 17ms/step - loss: 0.0144
Epoch 5/30
788/788 ━━━━━━━━━━━━━━━━━━━━ 20s 17ms/step - loss: 0.0126
Epoch 6/30
788/788 ━━━━━━━━━━━━━━━━━━━━ 20s 17ms/step - loss: 0.0112
Epoch 7/30
788/788 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step - loss: 0.0102
Epoch 8/30
788/788 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - loss: 0.0094
Epoch 9/30
788/788 ━━━━━━━━━━━━━━━━━━━━ 22s 19ms/step - loss: 0.0087
Epoch 10/30
788/788 ━━━━━━━━━━━━━━━━━━━━ 19s 17ms/step - loss: 0.0082
Epoch 11/30
788/788 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - loss: 0.0077
Epoch 12/30
788/788 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - loss: 0.0074
E

### Fine-tuning for each stock

In [14]:
MODEL_PATH = "trained_models/fine-tuned"
GENERAL_MODEL_PATH = "trained_models/hybrid_model.keras"

def fine_tune_model(stock_symbol):
    """Fine-tunes the hybrid model for a specific stock."""
    print(f"Fine-Tuning Model for {stock_symbol}...")

    # Load stock-specific data
    X, y, scaler = load_stock_data(stock_symbol=stock_symbol)

    # Load the pre-trained general model
    model = tf.keras.models.load_model("trained_models/hybrid_model.keras")

    # Ensure the final output layer is correctly reshaped
    if model.output_shape[-1] != 20 * 5:
        print("Updating output layer to match (20, 5)")
        x = model.layers[-2].output  # Get the last hidden layer before the output
        outputs = tf.keras.layers.Dense(20 * 5, activation="linear")(x)  # Correct output shape
        model = tf.keras.Model(inputs=model.input, outputs=outputs)

    # Freeze early layers to keep general knowledge
    for layer in model.layers[:-3]:
        layer.trainable = False

    # Recompile with a small learning rate for fine-tuning
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss="mean_squared_error")

    # Train on the specific stock data
    model.fit(X, y.reshape(-1, 20 * 5), epochs=10, batch_size=64)

    # Save the fine-tuned model
    fine_tuned_path = f"trained_models/fine_tuned/{stock_symbol}.keras"
    os.makedirs("trained_models/fine_tuned", exist_ok=True)
    model.save(fine_tuned_path)

    print(f"Fine-tuned Model saved at `{fine_tuned_path}`.")

# Fine-tune all stocks
for stock in POPULAR_STOCKS:
    fine_tune_model(stock)

print("Fine-tuning complete for all stocks.")

Fine-Tuning Model for AAPL...
Loaded 1008 samples for AAPL, Shape of X: (1008, 180, 12), Shape of y: (1008, 20, 5)
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0040
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0038
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0040
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0039
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0042
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0041
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0039
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0039
Epoch 9/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0038
Epoch 10/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0038
Fine-tuned Model saved at `trained_models/fine_tuned/AAPL.keras`.
Fine-Tuning Model for MSFT...
Loaded 1008 samples for MSFT, Shape of X: (1008, 180, 12), Shape of y: (1008, 20, 5)
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/s

### Evaluate models

In [15]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import json

In [17]:
def evaluate_predictions(y_true, y_pred):
    """Evaluate RMSE for each feature separately, ensuring proper shape."""
    feature_names = ["Open", "High", "Low", "Close", "Volume"]
    errors = {}

    # Ensure y_true and y_pred are correctly shaped as (samples, 20, 5)
    if y_true.ndim == 2:  # If missing last dimension, reshape
        y_true = y_true.reshape(-1, 20, 5)
    if y_pred.ndim == 2:
        y_pred = y_pred.reshape(-1, 20, 5)

    print(f"y_true shape: {y_true.shape}, y_pred shape: {y_pred.shape}")

    for i, feature in enumerate(feature_names):
        mse = mean_squared_error(y_true[:, :, i].flatten(), y_pred[:, :, i].flatten())  # Flatten arrays
        rmse = np.sqrt(mse)
        errors[feature] = rmse

    return errors

# Run evaluation
errors = evaluate_predictions(y_test, y_pred)
print("Feature-wise RMSE:", errors)

y_true shape: (1008, 20, 5), y_pred shape: (1008, 20, 5)
Feature-wise RMSE: {'Open': 0.05041280145736498, 'High': 0.04979242847137841, 'Low': 0.05108846753779055, 'Close': 0.04828601305963443, 'Volume': 0.07285282271985223}


In [23]:
def inverse_transform_predictions(y_pred, scaler):
    """Inverse transforms predictions using the saved MinMaxScaler."""
    num_samples, pred_days, num_features = y_pred.shape  # (1008, 20, 5)

    # Reshape to (samples * days, features) → (20160, 5)
    y_pred_reshaped = y_pred.reshape(-1, num_features)
    # Placeholder for 12 features
    temp = np.zeros((y_pred_reshaped.shape[0], 12))
    temp[:, :5] = y_pred_reshaped

    # Apply inverse transform
    temp_original = scaler.inverse_transform(temp)
    # Extract only the first 5 columns (since we only predicted these)
    y_pred_original = temp_original[:, :5]
    # Reshape back to (samples, 20, 5)
    return y_pred_original.reshape(num_samples, pred_days, num_features)

In [24]:
HYBRID_MODEL_PATH = "models/trained_models/hybrid/hybrid_model.keras"
FINE_TUNED_PATH = "models/trained_models/fine-tuned"
BEST_MODELS_PATH = "models/best_model_paths.json"

def evaluate_models(stock_symbol):
    """Evaluate RMSE for both general and fine-tuned models for a stock."""
    X_test, y_test, scaler = load_stock_data(stock_symbol=stock_symbol)

    # Load models
    general_model = tf.keras.models.load_model("trained_models/hybrid_model.keras")
    fine_tuned_model = tf.keras.models.load_model(f"trained_models/fine_tuned/{stock_symbol}.keras")

    # Make predictions
    y_pred_general = general_model.predict(X_test)
    y_pred_fine_tuned = fine_tuned_model.predict(X_test)

    # **Fix Output Shape**: Reshape predictions from (1008, 100) → (1008, 20, 5)
    y_pred_general = y_pred_general.reshape(-1, 20, 5)
    y_pred_fine_tuned = y_pred_fine_tuned.reshape(-1, 20, 5)

    # **Inverse Transform Predictions Correctly**
    y_pred_general = inverse_transform_predictions(y_pred_general, scaler)
    y_pred_fine_tuned = inverse_transform_predictions(y_pred_fine_tuned, scaler)
    y_test_original = inverse_transform_predictions(y_test, scaler)

    # **Compute RMSE for each model**
    def compute_rmse(y_true, y_pred):
        return np.sqrt(mean_squared_error(y_true.reshape(-1, 5), y_pred.reshape(-1, 5)))

    general_rmse = compute_rmse(y_test_original, y_pred_general)
    fine_tuned_rmse = compute_rmse(y_test_original, y_pred_fine_tuned)

    print(f"General Model RMSE for {stock_symbol}: {general_rmse}")
    print(f"Fine-Tuned Model RMSE for {stock_symbol}: {fine_tuned_rmse}")

    return "Fine-Tuned" if fine_tuned_rmse < general_rmse else "General"

# Run evaluation for all stocks
best_models = {stock: evaluate_models(stock) for stock in POPULAR_STOCKS}
print("Best model selection completed:", best_models)


Loaded 1008 samples for AAPL, Shape of X: (1008, 180, 12), Shape of y: (1008, 20, 5)


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
General Model RMSE for AAPL: 11439177.75471144
Fine-Tuned Model RMSE for AAPL: 11437389.882620467
Loaded 1008 samples for MSFT, Shape of X: (1008, 180, 12), Shape of y: (1008, 20, 5)


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
General Model RMSE for MSFT: 4420527.812925526
Fine-Tuned Model RMSE for MSFT: 4320237.099100187
Loaded 1008 samples for GOOGL, Shape of X: (1008, 180, 12), Shape of y: (1008, 20, 5)


...

32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
General Model RMSE for CAT: 614056.1370163984
Fine-Tuned Model RMSE for CAT: 607825.016383215
Loaded 1008 samples for MMM, Shape of X: (1008, 180, 12), Shape of y: (1008, 20, 5)


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
General Model RMSE for MMM: 2249017.978377782
Fine-Tuned Model RMSE for MMM: 2205236.277421034
Loaded 1008 samples for GS, Shape of X: (1008, 180, 12), Shape of y: (1008, 20, 5)


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step
General Model RMSE for GS: 519632.0395624065
Fine-Tuned Model RMSE for GS: 518213.29916695325
Best model selection completed: {'AAPL': 'Fine-Tuned', 'MSFT': 'Fine-Tuned', 'GOOGL': 'Fine-Tuned', 'AMZN': 'Fine-Tuned', 'TSLA': 'Fine-Tuned', 'META': 'Fine-Tuned', 'NVDA': 'Fine-Tuned', 'JPM': 'Fine-Tuned', 'JNJ': 'Fine-Tuned', 'V': 'Fine-Tuned', 'PG': 'Fine-Tuned', 'DIS': 'Fine-Tuned', 'MA': 'Fine-Tuned', 'PYPL': 'Fine-Tuned', 'NFLX': 'Fine-Tuned', 'ADBE': 'Fine-Tuned', 'INTC': 'Fine-Tuned', 'CMCSA': 'Fine-Tuned', 'PFE': 'Fine-Tuned', 'KO': 'Fine-Tuned', 'PEP': 'Fine-Tuned', 'CSCO': 'Fine-Tuned', 'XOM': 'Fine-Tuned', 'ABT': 'Fine-Tuned', 'CRM': 'Fine-Tuned', 'NKE': 'Fine-Tuned', 'MRK': 'Fine-Tuned', 'WMT': 'Fine-Tuned', 'T': 'Fine-Tuned', 'BAC': 'Fine-Tuned', 'MCD': 'Fine-Tuned', 'COST': 'Fine-Tuned', 'CVX': 'Fine-Tuned', 'MDT': 'Fine-Tuned', 'NEE': 'Fine-Tuned', 'LLY': 'Fine-Tuned', 'HON': 'Fine-Tuned', 'ORCL':

In [26]:
# Save best model paths
with open("trained_models/best_models.json", "w") as f:
    json.dump(best_models, f)

In [29]:
def inverse_transform_predictions(y_pred, scaler):
    """Inverse transforms predictions using the saved MinMaxScaler."""
    num_samples, pred_days, num_features = y_pred.shape  # (samples, 20, 5)

    # Reshape to (samples * days, features) → (20, 5)
    y_pred_reshaped = y_pred.reshape(-1, num_features)

    # **Fix: Create a placeholder array with the correct feature count (12)**
    temp = np.zeros((y_pred_reshaped.shape[0], 12))  # Placeholder for 12 features
    temp[:, :5] = y_pred_reshaped  # Fill first 5 columns with predictions

    # Apply inverse transform
    temp_original = scaler.inverse_transform(temp)

    # Extract only the first 5 columns (since we only predicted these)
    y_pred_original = temp_original[:, :5]

    # Reshape back to (samples, 20, 5)
    return y_pred_original.reshape(num_samples, pred_days, num_features)

In [32]:
def generate_predictions(stock_symbol):
    """Generate stock price predictions for a specific stock using the fine-tuned model."""
    X_test, _, scaler = load_stock_data(stock_symbol=stock_symbol)

    # Load the fine-tuned model
    fine_tuned_path = f"trained_models/fine_tuned/{stock_symbol}.keras"
    model = tf.keras.models.load_model(fine_tuned_path)

    # Generate predictions (scaled)
    y_pred_scaled = model.predict(X_test[-1:])

    # Ensure the output is reshaped correctly
    if y_pred_scaled.ndim == 2:
        y_pred_scaled = y_pred_scaled.reshape(-1, 20, 5)

    # Inverse transform predictions
    y_pred_original = inverse_transform_predictions(y_pred_scaled, scaler)

    return y_pred_scaled, y_pred_original

In [33]:
scaled_predictions, original_predictions = generate_predictions("AAPL")

Loaded 1008 samples for AAPL, Shape of X: (1008, 180, 12), Shape of y: (1008, 20, 5)


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step


In [35]:
original_predictions

array([[[2.29085211e+02, 2.28942422e+02, 2.30899699e+02, 2.27830755e+02,
         5.72839234e+07],
        [2.29334811e+02, 2.28870271e+02, 2.30985753e+02, 2.27783306e+02,
         5.77526822e+07],
        [2.29008702e+02, 2.28918985e+02, 2.31009440e+02, 2.27702041e+02,
         5.81274826e+07],
        [2.29025338e+02, 2.28839794e+02, 2.30840340e+02, 2.27675758e+02,
         5.75385879e+07],
        [2.28925905e+02, 2.28922916e+02, 2.30948315e+02, 2.27667142e+02,
         5.81551380e+07],
        [2.29085244e+02, 2.28859635e+02, 2.30807718e+02, 2.27856453e+02,
         5.84984297e+07],
        [2.29233735e+02, 2.28741218e+02, 2.30925849e+02, 2.27679649e+02,
         5.89341600e+07],
        [2.29090727e+02, 2.28790614e+02, 2.30868886e+02, 2.27770256e+02,
         5.85807578e+07],
        [2.28893646e+02, 2.28799029e+02, 2.30931514e+02, 2.27728768e+02,
         5.87365721e+07],
        [2.29190410e+02, 2.28898581e+02, 2.30930795e+02, 2.27684960e+02,
         5.82569023e+07],
        [2

## Real-time Predictions

In [15]:
from datetime import datetime, timedelta
import json

In [16]:
def fetch_latest_stock_data(stock_symbol):
    """Fetches the latest 180 days of stock data and applies technical indicators."""
    stock = yf.Ticker(stock_symbol)
    df = stock.history(period="1y")  # Fetch more data (ensure 180 days exist)

    if df.empty:
        raise ValueError(f"No recent data found for {stock_symbol}")

    # Keep only relevant columns
    df = df[['Close', 'Open', 'High', 'Low', 'Volume']]

    # Compute technical indicators
    df["SMA_20"] = ta.trend.sma_indicator(df["Close"], window=20)
    df["SMA_50"] = ta.trend.sma_indicator(df["Close"], window=50)
    df["EMA_20"] = ta.trend.ema_indicator(df["Close"], window=20)
    df["RSI"] = ta.momentum.rsi(df["Close"], window=14)
    df["MACD"] = ta.trend.macd(df["Close"])
    df["Bollinger_Upper"] = ta.volatility.bollinger_hband(df["Close"], window=20)
    df["Bollinger_Lower"] = ta.volatility.bollinger_lband(df["Close"], window=20)

    df.dropna(inplace=True)  # Drop NaN values

    # Ensure exactly 180 rows
    if len(df) < 180:
        raise ValueError(f"Not enough historical data for {stock_symbol}, only {len(df)} rows available.")

    return df[-180:]  # Return exactly 180 rows

In [22]:
def inverse_transform_predictions(y_pred, scaler):
    """Inverse transforms predictions using the saved MinMaxScaler."""
    num_samples, pred_days, num_features = y_pred.shape  # (samples, 20, 5)
    
    # Reshape from (samples, 20, 5) → (samples * days, 5)
    y_pred_reshaped = y_pred.reshape(-1, num_features)

    # Fix: Create a placeholder array for all 12 features
    temp = np.zeros((y_pred_reshaped.shape[0], 12))  # Placeholder for 12 features
    temp[:, :5] = y_pred_reshaped  # Fill first 5 columns with predictions

    # Apply inverse transform (restoring real values)
    temp_original = scaler.inverse_transform(temp)

    # Extract only the first 5 columns (Close, Open, High, Low, Volume)
    y_pred_original = temp_original[:, :5]

    # Reshape back to (samples, 20, 5)
    return y_pred_original.reshape(num_samples, pred_days, num_features)


In [17]:
def generate_predictions(stock_symbol):
    """Fetch latest stock data, make predictions, and return results."""
    latest_data = fetch_latest_stock_data(stock_symbol)  # Fetch live data

    # Ensure we have enough data
    if latest_data.shape[0] < SEQ_LENGTH:
        raise ValueError(f"Not enough data for {stock_symbol}. Expected 180 rows, got {latest_data.shape[0]}")

    X_input = latest_data.values[-SEQ_LENGTH:]  # Take last 180 days

    X_input = X_input.reshape(1, SEQ_LENGTH, 12)  # Ensure correct shape

    # Load best model for this stock
    with open("trained_models/best_models.json", "r") as f:
        best_models = json.load(f)

    best_model_type = best_models.get(stock_symbol, "General")
    model_path = f"trained_models/{'fine_tuned' if best_model_type == 'Fine-Tuned' else 'hybrid'}/{stock_symbol}.keras"

    model = tf.keras.models.load_model(model_path)

    # Generate predictions
    y_pred_scaled = model.predict(X_input)

    # Fix: Ensure correct reshaping
    if y_pred_scaled.ndim == 2:
        y_pred_scaled = y_pred_scaled.reshape(-1, 20, 5)  # Ensure (samples, 20, 5)

    # Use inverse transform function
    scaler_path = f"data_pipeline/scalers/{stock_symbol}_scaler.pkl"
    scaler = joblib.load(scaler_path)
    y_pred_original = inverse_transform_predictions(y_pred_scaled, scaler)

    return y_pred_original

In [18]:
def generate_overall_action(stock_symbol):
    """Generates a Buy, Sell, or Hold action based on predictions and indicators."""

    # Fetch latest stock data & make predictions
    latest_data = fetch_latest_stock_data(stock_symbol)
    predictions = generate_predictions(stock_symbol)

    # Extract close prices from predictions
    predicted_close = predictions[:, :, 3]  # Extract Close column (4th column)
    predicted_close = predicted_close[0]  # Take the first sequence

    # Compute percentage change over 5 days
    price_change_5d = (predicted_close[5] - predicted_close[0]) / predicted_close[0] * 100

    # Apply trading rules
    if price_change_5d > 5:
        action = "BUY"
    elif price_change_5d < -5:
        action = "SELL"
    else:
        action = "HOLD"

    print(f"{stock_symbol}: Predicted Close Change in 5 Days: {price_change_5d:.2f}% → Action: {action}")

    return action

In [23]:
overall_actions = {}
for stock in POPULAR_STOCKS:
    action = generate_overall_action(stock)
    overall_actions[stock] = action

print("\n✅ Overall Actions for All Stocks:")
for stock, action in overall_actions.items():
    print(f"{stock}: {action}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step


/opt/anaconda3/envs/stock_env/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


AAPL: Predicted Close Change in 5 Days: 7.93% → Action: BUY
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step


/opt/anaconda3/envs/stock_env/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


MSFT: Predicted Close Change in 5 Days: 7.23% → Action: BUY
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step


...

/opt/anaconda3/envs/stock_env/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


CAT: Predicted Close Change in 5 Days: 9.23% → Action: BUY
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step


/opt/anaconda3/envs/stock_env/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


MMM: Predicted Close Change in 5 Days: 4.35% → Action: HOLD
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
GS: Predicted Close Change in 5 Days: 10.73% → Action: BUY

✅ Overall Actions for All Stocks:
AAPL: BUY
MSFT: BUY
GOOGL: BUY
AMZN: BUY
TSLA: BUY
META: BUY
NVDA: BUY
JPM: BUY
JNJ: HOLD
V: HOLD
PG: HOLD
DIS: BUY
MA: BUY
PYPL: BUY
NFLX: BUY
ADBE: BUY
INTC: BUY
CMCSA: HOLD
PFE: HOLD
KO: HOLD
PEP: HOLD
CSCO: HOLD
XOM: BUY
ABT: HOLD
CRM: BUY
NKE: BUY
MRK: HOLD
WMT: BUY
T: HOLD
BAC: BUY
MCD: HOLD
COST: BUY
CVX: BUY
MDT: HOLD
NEE: HOLD
LLY: BUY
HON: HOLD
ORCL: BUY
AVGO: BUY
TXN: HOLD
UNH: HOLD
QCOM: BUY
BMY: HOLD
IBM: BUY
AMD: BUY
AMAT: BUY
GE: BUY
CAT: BUY
MMM: HOLD
GS: BUY


/opt/anaconda3/envs/stock_env/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


## Download files

In [83]:
!# Zip all trained models
!zip -r stock_trading_system.zip data_pipeline/ trained_models/ requirements.txt

  adding: data_pipeline/ (stored 0%)
  adding: data_pipeline/raw_data/ (stored 0%)
  adding: data_pipeline/raw_data/MA.csv (deflated 58%)
  adding: data_pipeline/raw_data/CMCSA.csv (deflated 58%)
  adding: data_pipeline/raw_data/AMZN.csv (deflated 64%)
  adding: data_pipeline/raw_data/WMT.csv (deflated 58%)
  adding: data_pipeline/raw_data/META.csv (deflated 57%)
  adding: data_pipeline/raw_data/INTC.csv (deflated 59%)
  adding: data_pipeline/raw_data/AMAT.csv (deflated 57%)
  adding: data_pipeline/raw_data/GOOGL.csv (deflated 58%)
  adding: data_pipeline/raw_data/DIS.csv (deflated 58%)
  adding: data_pipeline/raw_data/COST.csv (deflated 58%)
  adding: data_pipeline/raw_data/AAPL.csv (deflated 58%)
  adding: data_pipeline/raw_data/T.csv (deflated 59%)
  adding: data_pipeline/raw_data/CSCO.csv (deflated 58%)
  adding: data_pipeline/raw_data/ABT.csv (deflated 58%)
  adding: data_pipeline/raw_data/MCD.csv (deflated 58%)
  adding: data_pipeline/raw_data/MRK.csv (deflated 58%)
  adding: dat

In [85]:
from google.colab import files
files.download("stock_trading_system.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>